# ファイルのアップロード方法

## Google Driveにファイルをアップロード

Google Driveに`soturon`フォルダをそのままアップロードしてください。

**フォルダ構造**:
```
Google Drive/
└── MyDrive/
    └── soturon/
        ├── 自己教師あり学習/
        │   └── task/
        │       ├── __init__.py
        │       ├── dataset.py
        │       ├── model.py
        │       └── train.py
        ├── ノイズの付与(共通)/
        │   ├── __init__.py
        │   ├── add_noise.py
        │   ├── frequency_band_noise.py
        │   ├── localized_spike_noise.py
        │   └── amplitude_dependent_noise.py
        ├── eval.py
        └── data_lowF_noise.pickle
```



In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')


# タスク4: マスク予測 + 正則化項

自己教師あり学習によるノイズ検出


## 1. 環境設定


In [ ]:
# このセルは不要（セル1でマウント済み）


In [ ]:
# 必要なライブラリをインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install scikit-learn matplotlib


In [ ]:
# 作業ディレクトリを設定
import os
import sys

# Google Driveにsoturonフォルダをそのままアップロード
project_root = '/content/drive/MyDrive/soturon'  # soturonフォルダのルート
ssl_folder = f'{project_root}/自己教師あり学習'  # 自己教師あり学習フォルダ

if os.path.exists(ssl_folder):
    os.chdir(ssl_folder)
    sys.path.insert(0, ssl_folder)
    sys.path.insert(0, project_root)  # プロジェクトルートも追加
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {ssl_folder} が見つかりません。")
    print("Google Driveにsoturonフォルダを作成して、ファイルをアップロードしてください。")


## 2. データの確認


In [ ]:
# データファイルのパスを設定
pickle_path = '/content/drive/MyDrive/soturon/data_lowF_noise.pickle'

# データが存在するか確認
import os
if os.path.exists(pickle_path):
    print(f"✅ Data file found: {pickle_path}")
else:
    print(f"⚠️ Data file not found at {pickle_path}")
    print("Google Driveのsoturonフォルダにdata_lowF_noise.pickleをアップロードしてください。")


## 3. モデルのインポート


In [ ]:
# ファイルの存在確認
import os
import sys

# 作業ディレクトリを設定（重要！）
# Google Driveにsoturonフォルダをそのままアップロード
project_root = '/content/drive/MyDrive/soturon'  # soturonフォルダのルート
ssl_folder = f'{project_root}/自己教師あり学習'  # 自己教師あり学習フォルダ

if os.path.exists(ssl_folder):
    os.chdir(ssl_folder)
    sys.path.insert(0, ssl_folder)
    sys.path.insert(0, project_root)  # プロジェクトルートも追加（eval.pyやノイズモジュール用）
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {ssl_folder} が見つかりません。")
    print("Google Driveにsoturonフォルダを作成して、ファイルをアップロードしてください。")

# 必要なファイル/フォルダの存在確認
required_files = [
    f'{ssl_folder}/task/__init__.py',
    f'{ssl_folder}/task/dataset.py',
    f'{ssl_folder}/task/model.py',
    f'{ssl_folder}/task/train.py',
    f'{project_root}/ノイズの付与(共通)/__init__.py',
    f'{project_root}/ノイズの付与(共通)/add_noise.py',
    f'{project_root}/ノイズの付与(共通)/frequency_band_noise.py',
    f'{project_root}/ノイズの付与(共通)/localized_spike_noise.py',
    f'{project_root}/ノイズの付与(共通)/amplitude_dependent_noise.py',
    f'{project_root}/eval.py',
    f'{project_root}/data_lowF_noise.pickle'
]

missing_files = []
for file in required_files:
    if not os.path.exists(file):
        # パスを短縮して表示
        if file.startswith(ssl_folder):
            missing_files.append(file.replace(ssl_folder + '/', '自己教師あり学習/'))
        elif file.startswith(project_root):
            missing_files.append(file.replace(project_root + '/', ''))

if missing_files:
    print("⚠️ 以下のファイル/フォルダが見つかりません:")
    for file in missing_files:
        print(f"  - {file}")
    print(f"\n📁 Google Driveの {project_root} にファイルをアップロードしてください。")
    print("   フォルダ構造:")
    print("   soturon/")
    print("   ├── 自己教師あり学習/")
    print("   │   └── task/")
    print("   ├── ノイズの付与(共通)/")
    print("   ├── eval.py")
    print("   └── data_lowF_noise.pickle")
else:
    print("✅ すべての必要なファイルが見つかりました！")

# ノイズモジュールのパスを設定（dataset.pyがインポートする前に必要）
noise_module_path = os.path.join(project_root, 'ノイズの付与(共通)')
sys.path.insert(0, noise_module_path)
sys.path.insert(0, project_root)

# タスク4のモジュールをインポート
try:
    from task.dataset import Task4Dataset
    from task.model import Task4BERT
    from task.train import train_task4
    import eval
    print("✅ モジュールのインポートに成功しました！")
except ImportError as e:
    print(f"❌ モジュールのインポートに失敗しました: {e}")
    print("   ファイルが正しくアップロードされているか確認してください。")
    import traceback
    traceback.print_exc()


## 4. 学習の実行


In [ ]:
# デバイスの確認
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# チェックポイントを削除して最初から学習する場合（必要に応じて実行）
import os
checkpoint_path = "/content/drive/MyDrive/soturon/自己教師あり学習/task4_output/checkpoint.pth"
if os.path.exists(checkpoint_path):
    print("既存のチェックポイントが見つかりました。削除します...")
    os.remove(checkpoint_path)
    print("✓ チェックポイントを削除しました。最初から学習します。")
else:
    print("チェックポイントは存在しません。最初から学習します。")


In [ ]:
# 学習を実行
# 最初から学習する場合は resume=False に変更してください
# チェックポイントから再開する場合は resume=True のままにしてください

# モジュールのインポート（必要に応じて）
from task.train import train_task4

model, train_losses, val_losses, best_val_loss = train_task4(
    pickle_path=pickle_path,
    batch_size=16,  # 速度向上のため8→16に変更
    num_epochs=10,  # エポック数（10エポックで約1時間）
    lr=1e-3,  # 1e-4 → 1e-3に変更（正則化項の学習を促進）
    val_ratio=0.2,
    device=device,
    out_dir="/content/drive/MyDrive/soturon/自己教師あり学習/task4_output",  # 出力先をGoogle Driveに
    resume=False,  # Falseに変更: 最初から学習 / True: チェックポイントから再開
    lambda_reg=2.0,  # 正則化項の重みを増やす（0.5 → 2.0、ノイズ検知を重視）
    num_intervals=30,
    noise_type='frequency_band',  # ノイズタイプ: 'frequency_band', 'localized_spike', 'amplitude_dependent'
    noise_level=0.3,  # ノイズレベル（30%）
    margin=0.01,  # 0.2 → 0.01に変更（学習が進みやすくする）
)


In [ ]:
# 評価に必要なモジュールをインポート
import os
import sys
import importlib

# 作業ディレクトリを設定（重要！）
project_root = '/content/drive/MyDrive/soturon'  # soturonフォルダのルート
project_path = f'{project_root}/自己教師あり学習'  # 自己教師あり学習フォルダ
if os.path.exists(project_path):
    os.chdir(project_path)
    sys.path.insert(0, project_path)
    sys.path.insert(0, project_root)  # プロジェクトルートも追加（eval.pyやノイズモジュール用）
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {project_path} が見つかりません。")
    print("Google Driveにsoturonフォルダを作成して、ファイルをアップロードしてください。")

# 必要なモジュールをインポート
from task.dataset import Task4Dataset
from task.model import Task4BERT
import torch
import eval

# evalモジュールを再読み込み（eval.pyを変更した場合）
if 'eval' in sys.modules:
    importlib.reload(sys.modules['eval'])
    print("evalモジュールを再読み込みしました")

# デバイスの設定（まだ設定されていない場合）
if 'device' not in globals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

# データパスの設定（まだ設定されていない場合）
if 'pickle_path' not in globals():
    pickle_path = "/content/drive/MyDrive/soturon/data_lowF_noise.pickle"

# モデルの初期化（まだ初期化されていない場合）
if 'model' not in globals() or model is None:
    seq_len = 3000
    model = Task4BERT(
        seq_len=seq_len,
        d_model=64,
        n_heads=2,
        num_layers=2,
        dim_feedforward=128,
        dropout=0.1,
    ).to(device)
    print("モデルを初期化しました")


## 5. 評価の実行


In [ ]:
# テストデータで評価
from torch.utils.data import DataLoader

# テストデータセットを作成
test_dataset = Task4Dataset(
    pickle_path=pickle_path,
    num_intervals=30,
    noise_type='frequency_band',  # 学習時と同じノイズタイプ
    noise_level=0.3,  # 学習時と同じノイズレベル
    add_structured_noise_flag=True,  # 全体的な構造化ノイズを付与（学習時と同じ）
)

# テストデータローダーを作成（全データを使用）
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # 速度向上のため8→16に変更

# 最良のモデルを読み込み
best_model_path = "/content/drive/MyDrive/soturon/自己教師あり学習/task4_output/best_val_model.pth"
model.load_state_dict(torch.load(best_model_path, map_location=device))

# 評価を実行
results = eval.evaluate_model(
    model=model,
    dataloader=test_loader,
    method='self_supervised',
    device=device,
    num_intervals=30,
)

# 結果を表示
print("\n評価結果:")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1-score: {results['f1_score']:.4f}")
if 'loss' in results:
    print(f"Loss (CrossEntropyLoss): {results['loss']:.5f}")
if 'roc_auc' in results:
    print(f"ROC-AUC: {results['roc_auc']:.4f}")
if 'attention_diff' in results:
    print(f"アテンションウェイトの差: {results['attention_diff']:.4f}")
    print(f"  (正常区間 - ノイズ区間)")
if 'best_threshold' in results:
    print(f"最適閾値: {results['best_threshold']:.6f}")
if 'noise_attention_mean' in results:
    print(f"ノイズ区間の平均アテンション: {results['noise_attention_mean']:.6f}")
if 'normal_attention_mean' in results:
    print(f"正常区間の平均アテンション: {results['normal_attention_mean']:.6f}")


In [ ]:
# 混同行列を可視化
eval.plot_confusion_matrix(
    results['confusion_matrix'],
    title='タスク4: 混同行列',
    save_path="/content/drive/MyDrive/soturon/自己教師あり学習/task4_output/confusion_matrix.png"
)
print("混同行列を保存しました")


In [ ]:
# アテンションウェイトの分布を可視化
if 'attention_weights' in results:
    eval.plot_attention_distribution(
        results['attention_weights'],
        results['labels'],
        num_intervals=30,
        save_path="/content/drive/MyDrive/soturon/自己教師あり学習/task4_output/attention_distribution.png"
    )
    print("アテンションウェイトの分布を保存しました")
